In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from modules.dataset import Dataset, LABELS

2023-01-26 03:19:12.238465: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Our Utils

In [3]:
def get_y_true(data):
    y_true=[]
    for X,y in data:
        for label in y:
            y_true.append(label)
    y_true = tf.Variable(y_true)
    return y_true

In [4]:
def get_f1_score_model(NAME, fold_num, dataset):
    # Dataset
    train_dataset, test_dataset = dataset.get_kfold(fold_num, sample=False)

    # Modeling
    model = tf.keras.models.load_model("results/models/{}_fold_{}.h5".format(NAME, fold_num))

    # Label and Predict
    y_true = get_y_true(test_dataset)
    y_preds = model.predict(test_dataset)

    # m object
    m = tf.keras.metrics.AUC(multi_label=True, num_thresholds=200)
    m.update_state(y_true, y_preds)
    thresholds = m._thresholds

    # average threshold
    precision_average_list = []
    recall_average_list = []
    f1_thresholds_average_list = []
    true_positives, true_negatives, false_positives, false_negatives = m.variables
    for i_thresh, (tps, tns, fps, fns) in enumerate(zip(true_positives, true_negatives, false_positives, false_negatives)):
        f1_score_average = []
        precision_average = []
        recall_average = []
        for label, tp, tn, fp, fn in zip(LABELS, tps, tns, fps, fns):
            tp, tn, fp, fn = tp.numpy(), tn.numpy(), fp.numpy(), fn.numpy()
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f1_score = 2 * tp / (2 * tp + fn + fp)
            precision_average.append(precision)
            recall_average.append(recall)
            f1_score_average.append(f1_score)

        precision_score_average = np.mean(precision_average)
        precision_average_list.append(precision_score_average)
        
        recall_score_average = np.mean(recall_average)
        recall_average_list.append(recall_score_average)
        
        f1_score_average = np.mean(f1_score_average)
        f1_thresholds_average_list.append((thresholds[i_thresh], f1_score_average))

    # Cast to numpy for vectorlize
    f1_thresholds_average_list = np.array(f1_thresholds_average_list)
    best_f1 = f1_thresholds_average_list.max(axis=0)[1]
    best_f1_indx = f1_thresholds_average_list.argmax(axis=0)[1]
    best_threshold = f1_thresholds_average_list.max(axis=0)[0]
    
    print("Thresholds: ")
    print(best_threshold)
    
    return precision_average_list[best_f1_indx], recall_average_list[best_f1_indx], best_f1

### Exp 1 : DenseNet121_None

In [ ]:
# Settings
NAME = "DenseNet121_None"
dataset = Dataset()

precision_score_list = []
recall_score_list = []
f1_score_list = []
for fold_num in range(1, 5+1):
    precision_score, recall_score, f1_score = get_f1_score_model(NAME, fold_num, dataset)
    precision_score_list.append(precision_score)
    recall_score_list.append(recall_score)
    f1_score_list.append(f1_score)

2023-01-26 03:22:03.004840: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-01-26 03:22:04.231790: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-26 03:22:04.233771: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-26 03:22:04.233826: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-01-26 03:22:04.235020: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-26 03:22:04.235137: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


      5/Unknown - 12s 38ms/step

2023-01-26 03:22:11.151552: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1402/1402 [==============================] - 71s 42ms/step
Thresholds: 
1.0000001
1402/1402 [==============================] - 56s 39ms/step
Thresholds: 
1.0000001
1402/1402 [==============================] - 55s 38ms/step
Thresholds: 
1.0000001


In [ ]:
print("Best={:.4f}, Average={:.4f}, Worst={:.4f}".format(max(f1_score_list), 
                                                         np.mean(f1_score_list),
                                                         min(f1_score_list)))

In [ ]:
precision_score_list, recall_score_list, f1_score_list

### Exp 2: DenseNet121_imagenet

In [49]:
# Settings
NAME = "DenseNet121_imagenet"
dataset = Dataset()

f1_score_list = []
for fold_num in range(1, 5+1):
    f1_score = get_f1_score_model(NAME, fold_num, dataset)
    f1_score_list.append(f1_score)

1402/1402 [==============================] - 57s 39ms/step


In [50]:
print("Best={:.4f}, Average={:.4f}, Worst={:.4f}".format(max(f1_score_list), 
                                                         np.mean(f1_score_list),
                                                         min(f1_score_list)))

Best=0.2215, Average=0.2151, Worst=0.2074


In [ ]:
f1_score_list

### Exp 3: EfficientNetB0_None

In [40]:
# Settings
NAME = "EfficientNetB0_None"
dataset = Dataset()

f1_score_list = []
for fold_num in range(1, 5+1):
    f1_score = get_f1_score_model(NAME, fold_num, dataset)
    f1_score_list.append(f1_score)

1402/1402 [==============================] - 42s 29ms/step


In [41]:
print("Best={:.4f}, Average={:.4f}, Worst={:.4f}".format(max(f1_score_list), 
                                                         np.mean(f1_score_list),
                                                         min(f1_score_list)))

Best=0.2263, Average=0.2204, Worst=0.2159


In [ ]:
f1_score_list

### Exp 4: EfficientNetB0_imagenet

In [43]:
# Settings
NAME = "EfficientNetB0_imagenet"
dataset = Dataset()

f1_score_list = []
for fold_num in range(1, 5+1):
    f1_score = get_f1_score_model(NAME, fold_num, dataset)
    f1_score_list.append(f1_score)

1402/1402 [==============================] - 42s 29ms/step


In [44]:
print("Best={:.4f}, Average={:.4f}, Worst={:.4f}".format(max(f1_score_list), 
                                                         np.mean(f1_score_list),
                                                         min(f1_score_list)))

Best=0.1765, Average=0.1496, Worst=0.1354


In [ ]:
f1_score_list

### Exp 5: Resnet50_None

In [45]:
# Settings
NAME = "Resnet50"
dataset = Dataset()

f1_score_list = []
for fold_num in range(1, 5+1):
    f1_score = get_f1_score_model(NAME, fold_num, dataset)
    f1_score_list.append(f1_score)

1402/1402 [==============================] - 38s 27ms/step


In [46]:
print("Best={:.4f}, Average={:.4f}, Worst={:.4f}".format(max(f1_score_list), 
                                                         np.mean(f1_score_list),
                                                         min(f1_score_list)))

Best=0.2008, Average=0.1963, Worst=0.1895


In [ ]:
f1_score_list

### Exp 6: Resnet50_imagenet

In [6]:
# Settings
NAME = "Resnet50_imagenet"
dataset = Dataset()

f1_score_list = []
for fold_num in range(1, 5+1):
    precision, recall, f1_score = get_f1_score_model(NAME, fold_num, dataset)
    f1_score_list.append(f1_score)

1402/1402 [==============================] - 37s 26ms/step


In [7]:
print("Best={:.4f}, Average={:.4f}, Worst={:.4f}".format(max(f1_score_list), 
                                                         np.mean(f1_score_list),
                                                         min(f1_score_list)))

Best=0.2087, Average=0.2065, Worst=0.2028


In [8]:
f1_score_list

[0.20717022636086754,
 0.20821138673046882,
 0.20568500260137842,
 0.2028188009153436,
 0.2086717115430576]